Adaptado de https://towardsdatascience.com/diy-ai-an-old-school-matrix-nn-401a00021a55

Mas usando gradiente descendente para o treino (https://levelup.gitconnected.com/training-a-single-perceptron-405026d61f4b)

In [21]:
!pip install terminaltables

import urllib.request

import numpy
import os
import random
import datetime
from terminaltables import AsciiTable


from string import punctuation
from os import listdir
from collections import Counter

from sklearn.model_selection import train_test_split

from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras import layers


In [22]:
# we need to tell numpy the dimensions of our arrays
samples = numpy.empty([0, 256])
results = numpy.empty([0, 10])

url = 'https://raw.githubusercontent.com/matheusvanzan/ia-datasets/master/semeion.txt'

for line in urllib.request.urlopen(url):
    # split line to array using space as separator
    numbers = line.decode('utf-8').split(' ')
    # as line read from the file is always is string, we need to convert first 256 parts to decimals,
    # and following 10 to integers
    sample = numpy.array([ float(x) for x in numbers[0:256] ])
    result = numpy.array([ int(x) for x in numbers[256:266] ])

    # print(samples.ndim, numpy.array([sample]).ndim) # 2 2
    # print(samples.shape, numpy.array([sample]).shape) # (0, 256) (1, 256)
    # print(type(samples), type(sample))

    # after that, append freshly read sample and result to arrays
    samples = numpy.concatenate( (samples, numpy.array([sample])), axis=0)
    results = numpy.concatenate((results, numpy.array([result])), axis=0)


print(samples, samples.shape)
print(results, results.shape)

X = samples
y = results

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (1593, 256)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]] (1593, 10)


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(1274, 256) (319, 256)
(1274, 10) (319, 10)


In [27]:
start = datetime.datetime.now()

# define network
model = Sequential([
    layers.Input(shape=(256,)),
    layers.Dense(units=256, activation='sigmoid'),
    layers.Dense(units=10, activation='sigmoid')
])

# compile network
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

# fit network
model.fit(X_train, y_train, epochs=200, verbose=None)

# evaluate
loss, acc = model.evaluate(X_test, y_test, verbose=1)
print('Test Accuracy: %f' % (acc*100))

stop = datetime.datetime.now()
print(str(stop-start).split('.')[0])

10/10 [==============================] - 0s 4ms/step - loss: 0.2288 - accuracy: 0.7680
Test Accuracy: 76.802510
0:00:19


In [25]:
import pandas

# computes result from [1x256] sample, requires first_layer and second_layer to be defined globally
# returns single detected number
def compute_result(model, input_sample):
    # process input vector through both layers on NN
    X = numpy.array([input_sample])
    prediction = model.predict(X)

    df = pandas.DataFrame(prediction)
    return numpy.argmax(df.values)

# converts [1x256] sample line into pretty 16x16 character block where 1 is * and other symbols are omitted
def print_sample(input_sample):
    # convert [1x256] matrix to [16x16]
    input_sample = input_sample.reshape(16, 16).tolist()

    text = []

    # process sample row by row
    for sample_row in range(16):
        text_row = input_sample[sample_row]
        # replace 1 with * and 0 with empty space
        text_row = map(lambda cell: '*' if cell == 1 else ' ', text_row)
        # join 16 characters into line
        text_row = ''.join(text_row)
        # line to rows array
        text.append(text_row)

    # finally, join rows with newlines
    return '\n'.join(text)



In [26]:
print('Actual testing of trained NN')

table_data = [
    ['Sample', 'Digit', 'Sample', 'Digit', 'Sample', 'Digit', 'Sample', 'Digit']
]

# we print three rows
for row in range(3):
    table_data.append([''] * 8)
    # with 8 columns, 4 image -> result pairs
    for col in range(4):
        # pick one random sample between 0 and sample count
        ri = random.randint(0, samples.shape[0] - 1)
        sample = samples[ri]

        table_data[row+1][col*2] = print_sample(sample)
        table_data[row+1][col*2+1] = '\n'.join([' ' * 5, ' ' * 5, '  %d' % compute_result(model, sample)])

table = AsciiTable(table_data)
table.inner_row_border = True

print(table.table)

Actual testing of trained NN
1/1 [==============================] - 0s 18ms/step
+------------------+-------+------------------+-------+------------------+-------+------------------+-------+
| Sample           | Digit | Sample           | Digit | Sample           | Digit | Sample           | Digit |
+------------------+-------+------------------+-------+------------------+-------+------------------+-------+
|            ***** |       |        ********  |       |         *******  |       |           ****** |       |
|           ****** |       |     ******   **  |       |        ***   *** |       | **************** |       |
|           *** ** |   1   |   ***        **  |   9   |        **    **  |   1   | *********   ***  |   7   |
|           *** ** |       | ***          **  |       |       ***   ***  |       |            ****  |       |
|           *** ** |       | *****       **   |       |       ***   **   |       |            **    |       |
|           *** ** |       |           